In [33]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Input, concatenate


In [9]:
train = pd.read_csv('../data/RFMiD_Training_Labels.csv').set_index('ID')
test = pd.read_csv('../data/RFMiD_Testing_Labels.csv').set_index('ID')
eval = pd.read_csv('../data/RFMiD_Validation_Labels.csv').set_index('ID')


In [10]:
train.columns


Index(['Disease_Risk', 'DR', 'ARMD', 'MH', 'DN', 'MYA', 'BRVO', 'TSLN', 'ERM',
       'LS', 'MS', 'CSR', 'ODC', 'CRVO', 'TV', 'AH', 'ODP', 'ODE', 'ST',
       'AION', 'PT', 'RT', 'RS', 'CRS', 'EDN', 'RPEC', 'MHL', 'RP', 'CWS',
       'CB', 'ODPM', 'PRH', 'MNF', 'HR', 'CRAO', 'TD', 'CME', 'PTCR', 'CF',
       'VH', 'MCA', 'VS', 'BRAO', 'PLQ', 'HPED', 'CL'],
      dtype='object')

In [36]:
X_train = train.drop(columns='Disease_Risk')
y_train = train['Disease_Risk']
X_eval  = eval.drop(columns='Disease_Risk')
y_eval = eval['Disease_Risk']


In [16]:
X_train.sample()


,DR,ARMD,MH,DN,MYA,BRVO,TSLN,ERM,LS,MS,CSR,ODC,CRVO,TV,AH,ODP,ODE,ST,AION,PT,RT,RS,CRS,EDN,RPEC,MHL,RP,CWS,CB,ODPM,PRH,MNF,HR,CRAO,TD,CME,PTCR,CF,VH,MCA,VS,BRAO,PLQ,HPED,CL
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1067,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
labels = train['Disease_Risk'].values


In [28]:
def load_and_preprocess_image(image_id, image_folder, target_size=(224, 224)):
    """
    Load and preprocess an image given its ID.
    """
    image_path = os.path.join(image_folder, f'{image_id}.png')
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = image / 255.0  # Normalize to [0, 1]
    return image


In [30]:
image_folder = '../data/training_images'
images = np.array([load_and_preprocess_image(row_id, image_folder) for row_id in X_train.index])


In [37]:
eval_image_folder = '../data/eval_images'
eval_images = np.array([load_and_preprocess_image(row_id, image_folder) for row_id in X_eval.index])


In [35]:
image_input = Input(shape=(224, 224, 3))
x = Conv2D(32, (3, 3), activation='relu')(image_input)
x = MaxPooling2D(2, 2)(x)
x = Flatten()(x)


row_input = Input(shape=(X_train.shape[1],))
y = Dense(64, activation='relu')(row_input)

# Combine the outputs of the two branches
combined = concatenate([x, y])

# Add a fully connected layer and the output layer
z = Dense(64, activation='relu')(combined)
z = Dense(1, activation='sigmoid')(z)  # Use 'softmax' if it's a multi-class problem

# Create a model with two inputs and one output
model = Model(inputs=[image_input, row_input], outputs=z)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [39]:
model.fit(
    [images, X_train], y_train,
    validation_data=([eval_images, X_eval], y_eval),
    epochs=5,
    batch_size=32
)


Epoch 1/5
60/60 [==============================] - 14s 225ms/step - loss: 0.0220 - accuracy: 0.9990 - val_loss: 0.0486 - val_accuracy: 0.9891
Epoch 2/5
22/60 [==========>...................] - ETA: 8s - loss: 0.0176 - accuracy: 0.9972

KeyboardInterrupt: 

In [40]:
model.evaluate([eval_images, X_eval], y_eval)


20/20 [==============================] - 1s 55ms/step - loss: 0.0417 - accuracy: 0.9937


[0.04168587177991867, 0.9937499761581421]